### First follow the steps of preprocess.ipynb and then run this file. 

In [ ]:
TRAIN_PATH = "CovidDataset/Train"
VAL_PATH = "CovidDataset/Test"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import * 
from keras.preprocessing import image

In [ ]:
# CNN Based Model in Keras

model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 54, 54, 64)       

In [ ]:
# Train from scratch
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_dataset = image.ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    'CovidDataset/Train',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')


Found 224 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [ ]:
validation_generator = test_dataset.flow_from_directory(
    'CovidDataset/Val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')

Found 60 images belonging to 2 classes.


In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=8,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps=2
)

Epoch 1/10
8/8 [==============================] - 14s 2s/step - loss: 0.9442 - accuracy: 0.5508 - val_loss: 0.6861 - val_accuracy: 0.5000
Epoch 2/10
8/8 [==============================] - 8s 1s/step - loss: 0.5456 - accuracy: 0.7305 - val_loss: 0.3394 - val_accuracy: 0.9167
Epoch 3/10
8/8 [==============================] - 9s 1s/step - loss: 0.4338 - accuracy: 0.7969 - val_loss: 0.3315 - val_accuracy: 0.9500
Epoch 4/10
8/8 [==============================] - 9s 1s/step - loss: 0.2984 - accuracy: 0.8828 - val_loss: 0.2860 - val_accuracy: 0.9000
Epoch 5/10
8/8 [==============================] - 9s 1s/step - loss: 0.2308 - accuracy: 0.9141 - val_loss: 0.0336 - val_accuracy: 0.9833
Epoch 6/10
8/8 [==============================] - 9s 1s/step - loss: 0.1729 - accuracy: 0.9336 - val_loss: 0.1870 - val_accuracy: 0.9667
Epoch 7/10
8/8 [==============================] - 9s 1s/step - loss: 0.1821 - accuracy: 0.9414 - val_loss: 0.1060 - val_accuracy: 0.9667
Epoch 8/10
8/8 [========================

Evaluate model

In [ ]:
model.save('model_adv.h5')

In [ ]:
model.evaluate_generator(train_generator)

In [ ]:
model.evaluate_generator(validation_generator)

Test with images

In [ ]:
model = load_model('model_adv.h5')

In [ ]:
import os

In [ ]:
train_generator.class_indices

In [ ]:
y_actual = []
y_test = []

In [ ]:
for i in os.listdir('./CovidDataset/Test/Normal/'):
    img = image.load_img('./CovidDataset/Test/Normal/' + i, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    p = model.predict_classes(img)
    y_test.append(p[0, 0])
    y_actual.append(1)

In [ ]:
for i in os.listdir('./CovidDataset/Test/Covid/'):
    img = image.load_img('./CovidDataset/Test/Covid/' + i, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    p = model.predict_classes(img)
    y_test.append(p[0, 0])
    y_actual.append(1)

In [ ]:
y_actual = np.array(y_actual)
y_test = np.array(y_test)


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(y_actual, y_test)

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(cm, cmap='plasma', annot=True)